## Introduce

Hello!

This competition is my first competition.
After studying some notebooks of this competition, I summarized the overall basic process of regression analysis.

The table of contents is as follows.

1. Data search:

    1) Univariate search:
        
    2) Bivariate search:

2. Pre-processing.

    1) Cleansing - Outliers..
                        
    2) Cleansing - Processing missing values.

    3) Derivative variable generation
    
    4) Change variables.

    5) Dummyization.

3. Modeling.

    1) Create a basic model.
    
    2) HyperParameter Tuning: Pipeline + GridSearch(+Optuna) + Cross Validation
       
    3) Model learning.
    
    4) Ensemble: Stacking and blending.
    
    5) The final result.

## 1. Data search

In [21]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings(action='ignore')


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8")) #check the files available in the directory

In [22]:
# read dataset
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [23]:
train.head()

In [24]:
test.head()

#### 1) Univariate search


It is very important to understand the meaning, type, and shape of each variable.

So I simply divided it into numeric variables and other variables.

I used the '_get_numerical_data' function of numpy to find out the numeric variable.

In [25]:
# numeric vars
num_cols = set(train._get_numeric_data().columns)

In [26]:
# other vars
other_cols = set(train.columns) - set(num_cols)


You can also think of numeric variables as continuous variables and other variables as categorical variables.

However, there may be categorical variables among numeric variables.

I thought the most obvious way to distinguish them is to read the variable manual and distinguish them directly.

In [27]:
# nominal vars
nominal_vars = [
    'MSZoning', 'LandContour', 'Utilities',
    'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
    'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 
    'GarageType', 'MiscFeature', 
    'SaleType', 'SaleCondition'
]

In [28]:
# ranking vars
ranking_vars = [
    'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
    'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'Street', 'Alley',
    'LandSlope', 'Functional', 'GarageFinish', 'MoSold', 'YrSold', 'PavedDrive', 
    'CentralAir', 'LotShape', 'MSSubClass', 
]

In [29]:
# continuous vars
continue_vars = [
    'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
    'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
    'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'BedroomAbvGr', 'KitchenAbvGr',
    'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'GarageYrBlt', 'YearBuilt', 'YearRemodAdd',
    'OverallQual'
]


This time, we checked basic statistics for continuous variables.

I found that the minimum value of some variables is zero.

In [30]:
train[continue_vars].describe()


After that, I explored the distribution of each variable.

First, we checked the dependent variable 'SalePrice'.

In [31]:
print(f'skew: {train.SalePrice.skew()}')
print(f'kurt: {train.SalePrice.kurt()}')
sns.distplot(train.SalePrice, fit=norm)
f = plt.figure()
stats.probplot(train.SalePrice, plot=plt)
plt.show()

"skew" means skewness and "kurt" means kurtosis.

Skewness represents the bias of the distribution, and kurtosis represents the sharpness of the distribution.

I thought about why to check these indicators and concluded that it was because it was an indicator that could judge the normality of the variable.

Variables, especially dependent variables, must follow normality to ensure that the model performs well.
The absolute value of skewness is 0-3 and kurtosis is 1-8.

The histogram is a good chart to check the distribution of variables.

The qq plot is a good chart for determining normality, and the point must follow the red diagonal.

Currently, it is difficult to judge that the dependent variable is perfectly normal.

In [32]:
train.SalePrice = np.log1p(train.SalePrice)

print(f'skew: {train.SalePrice.skew()}')
print(f'kurt: {train.SalePrice.kurt()}')
sns.distplot(train.SalePrice, fit=norm)
f = plt.figure()
stats.probplot(train.SalePrice, plot=plt)
plt.show()

In this case, log conversion is a solution.

Log transformation allows variables to follow normality.

In [33]:
X_total = pd.concat((train, test)).drop(['SalePrice'], axis=1)

Draw a histogram of continuous variables.

In [34]:
f, ax = plt.subplots(6, 6, figsize=(25, 20))
for i, c in enumerate(continue_vars):
    sns.distplot(X_total[c], fit=stats.norm, ax=ax[i//6, i%6])

Some variables seem to be able to follow a normal distribution through log transformation or box cox transformation.
Some variables have zero. It seems that +1 should be done when converting.

Draw a bar chart for categorical variables.

Some variables were extremely biased toward one value (0). So they don't seem to be very important variables.

In [35]:
f, ax = plt.subplots(5, 5, figsize=(25, 15))
for i, c in enumerate(nominal_vars):
    g = sns.barplot(data=pd.DataFrame(X_total[c].value_counts()).reset_index(), x='index', y=c, ax=ax[i//5, i%5])
    g.set(xticks=[])
    g.set(title=c)

#### 2) Bivariate search


Now, we have explored the two variables.

First, a correlation analysis was conducted to confirm the linear relationship between variables.

The 'corr' function helps you do this.

Seaborn's heatmap is a chart that helps visualize the results of correlation analysis.

In [36]:
f, ax = plt.subplots(figsize=(10, 8))
corr = train.corr()
sns.heatmap(corr)
plt.show()

If you look at the heat map, you can judge that it has a high correlation if it is close to white or black.

I paid attention to variables that have a high correlation with the dependent variable.

Our purpose is to match the value of the dependent variable, and I thought that variables that have a high correlation with the dependent variable would have important characteristics to solve the problem.

In [37]:
f, ax = plt.subplots(figsize=(8, 7))
top_corr = train[corr.SalePrice.sort_values(ascending=False)[:10].index].corr()
sns.heatmap(top_corr, annot=True)
plt.show()

I selected only the top 10 variables with the highest correlation with the dependent 
variable and conducted the correlation analysis again.

I selected only the top 10 variables with the highest correlation with the dependent variable and conducted the correlation analysis again.

"OverallQual" and "GrLiv Area" are the strongest variables.

Garage Cars and Garage Area are also highly correlated. High correlation between independent variables is not good because it causes multicollinearity.



Among them, I drew a box plot and scatterplot using 'OverallQual' and 'GrLiv Area', which show the highest correlation with the dependent variable.

I understand that they have a strong linear relationship with the dependent variable. So I had an expectation that I could find an outlier.

In [38]:
sns.boxplot(data=train[['SalePrice', 'OverallQual']], x='OverallQual', y='SalePrice')
plt.show()

In [39]:
train[['SalePrice', 'GrLivArea']].plot.scatter(x='SalePrice', y='GrLivArea')
plt.show()

Oh! Let's look at the scatter plot with GrLiv Area. 

We can see that the two points with the largest GrLiv Area are completely out of the linear relationship.

Draw pair plot!

In [40]:
sns.pairplot(train[corr.SalePrice.sort_values(ascending=False)[:10].index])
plt.show()

## 2. Pre-processing

#### 1) Cleansing - Outliers

First, I removed the "ID" that is not an outlier but is not needed for analysis.

In [41]:
# delete ID
train_id = train.Id
test_id = test.Id
test_idx = test.index
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

Next, the two outliers identified in the scatter plot were removed.

In [42]:
train.drop(train.GrLivArea.sort_values(ascending=False)[:2].index, axis=0, inplace=True)

Let's draw a scatterplot again. It's clean.

In [43]:
train[['SalePrice', 'GrLivArea']].plot.scatter(x='SalePrice', y='GrLivArea')
plt.show()

#### 2) Cleansing - Processing missing values

Log conversion of dependent variables and removal of outliers in training data are over.

There is no need to stay in the training data anymore.

Therefore, we separate the dependent variable and combine training X and test X.

In [44]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train['SalePrice'].values
all_data = pd.concat((train, test), axis=0).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)

Create a missing table using the 'isnull' function.

In [45]:
count = all_data.isnull().sum()
percent = (count / all_data.isnull().count() * 100).sort_values(ascending=False)
missing_table = pd.DataFrame({'percent': percent})
missing_table.head(10)

In [46]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=percent.index[:30], y=percent[:30])
plt.title('missing percent', fontsize=15)
plt.xlabel('features', fontsize=15)
plt.ylabel('percent', fontsize=15)
plt.show()

Criteria for processing missing values include deletion and replacement.

'Delete method' deletes the columns in which missing values exist in the simplest and cleanest way.

In this competition, Rows should not be deleted because missing values may exist in the test data.

The condition that a column can be deleted is that more than 15% of the column is missing or does not affect the analysis at all.

replacements is an option that should be considered before deletion.

Simple replacements are again specific value replacements and mean value replacements (average, median, mod, regression), and simple probability replacements (hot deck, cold deck).

'specific value replacements' is possible when there is background knowledge that missing values have some meaning.

It can be applied mainly to categorical variables. ex) All other values are recorded, but only specific values are recorded.

In this case, these values usually correspond to special cases or negative meanings or
it may be a value that may be reluctant to check.

In this case, the rate of missing values is usually abnormally high. Therefore, if you read the explanation of the variable and fall under the above situation,
It replaces it with a specific value.

The mean value replacement is applied to continuous variables and is used when missing values can be inferred through values of other variables.

In [47]:
# specific value replacements
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")


All of the above missing values were replaced with specific values.
The reason is that we found that their missing values mean Nan or some specific value.


In [48]:
# mean value replacements
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

The above missing values are continuous variables and have been replaced with the poorest values.

Lot Frontage' will be similar to the value of nearby houses, so it has been replaced with a median value.

In [49]:
all_data.drop(['Utilities'], axis=1, inplace=True)

'Utilities', 'Condition2', 'RoofMatl', 'Heating' are biased variables. 

They are removed because they are variables that did not significantly affect the analysis.

GarageArea, 1stFlrSF are also removed (High correlation coefficient.).

In [50]:
# Complete
all_data.isnull().sum().sum()

#### 3) Derivative variable generation


Derivative variables are methods that can improve the quality of analysis.
There are several ideas for how to generate derivative variables.

1. It is a continuous variable and means an observation of an object, and if this value is 0, it means that there is no object.

Therefore, it is possible to add categorical variables as to whether or not the object is present.
Categorical variables with binary values can be stored as 0 and 1.

2. If you can express continuous variables of the same series in association, use a four-line operation.

New variables can be created. It is similar to calculating BMI with height and weight.

The above judgment can be used using min and max values after describe.

In [51]:
all_data.describe().T[(all_data.describe().T)['min'] == 0].index

In [52]:
all_data['HasMasVnr'] = all_data.MasVnrArea.apply(lambda x: 1 if x else 0)
all_data['Has2ndFlrSF'] = all_data['2ndFlrSF'].apply(lambda x: 1 if x else 0)
all_data['HasGarageArea'] = all_data['GarageArea'].apply(lambda x: 1 if x else 0)
all_data['HasWoodDeckSF'] = all_data['WoodDeckSF'].apply(lambda x: 1 if x else 0)
all_data['HasOpenPorchSF'] = all_data['OpenPorchSF'].apply(lambda x: 1 if x else 0)
all_data['HasEnclosedPorch'] = all_data['EnclosedPorch'].apply(lambda x: 1 if x else 0)
all_data['Has3SsnPorch'] = all_data['3SsnPorch'].apply(lambda x: 1 if x else 0)
all_data['HasScreenPorch'] = all_data['ScreenPorch'].apply(lambda x: 1 if x else 0)
all_data['HasPoolArea'] = all_data['PoolArea'].apply(lambda x: 1 if x else 0)
all_data['HasMiscVal'] = all_data['MiscVal'].apply(lambda x: 1 if x else 0)

all_data['TotalBath'] = all_data['BsmtFullBath'] + all_data['BsmtHalfBath'] * 0.5\
+ all_data['FullBath'] + all_data['HalfBath'] * 0.5
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
#all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['2ndFlrSF']
all_data['TotalBsmtSF'] = all_data['TotalBsmtSF'] + all_data['BsmtFinSF1'] \
+ all_data['BsmtFinSF2']

#### 4) Change variables

Categorical variables must proceed with 'dummy' work.

The function used at this time is 'get_dummies'. 

This function should be applied to all categorical variables. 

To do this, all categorical variables must be 'str'. Therefore, the types of numerical categorical variables are changed earlier.

In [53]:
nominal_vars = list(set(nominal_vars) - set(['Utilities']))

In [54]:
all_data[nominal_vars] = all_data[nominal_vars].astype(str)
all_data[ranking_vars] = all_data[ranking_vars].astype(str)

Among categorical variables, there are variables indicating order or ranking.

They perform label encoding instead of one hot encoding. Then, the meaning of ranking can be saved.

The following variables correspond to this.

In [55]:
from sklearn.preprocessing import LabelEncoder

for c in ranking_vars:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

It is advantageous for analysis that all continuous variables follow normality.

We select variables with high skewness values.

In [56]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head()

We're going to change the boxcox for the variables selected above.
Reduce skewness.

In [57]:
skewness = skewness[abs(skewness) > 0.75]

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)

#### 5) Dummyization

For all categorical types (excluding order variables), get_dummies, which automatically allow one-hot encoding, are applied.

In [58]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

## 3. Modeling

#### 1) Create a basic model

Create basic models for analysis.

Use lasso, elasticNet, KernelRidge, GradientBoosting, XGBoost, LightGBM

In [59]:
X_train = all_data[:ntrain]
X_test = all_data[ntrain:]

In [60]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as rfr, GradientBoostingRegressor as gbr
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor

import optuna
from functools import partial

Create a function that calculates the cross validation score.

In [61]:
n_folds = 5

def rmsle(model, x, y):
    return np.sqrt(mean_squared_error(y, model.predict(x)))

def rmsle_cv(model, x, y):
    #kf = KFold(n_splits=5, shuffle=True, random_state=42)
    kf = 5
    rmse =np.sqrt(-cross_val_score(model, x.values, y, scoring='neg_mean_squared_error', cv=kf))
    return rmse

#### 2) HyperParameter Tuning: Pipeline + GridSearch(+Optuna) + Cross Validation

Build pipelines for Laso, ElasticNet, and Kernel Ridge and parameters to use for grid search.

In [62]:
pipe_lasso = Pipeline([
    ('scaler', RobustScaler()),
    ('model', Lasso())
])
pipe_enet = Pipeline([
    ('scaler', RobustScaler()),
    ('model', ElasticNet(max_iter=5000))
])
pipe_krr = Pipeline([
    ('scaler', RobustScaler()),
    ('model', KernelRidge())
])

grid_param_lasso = [{
    'model__alpha': 0.0001 * np.arange(1, 100)
}]
grid_param_enet = [{
    'model__alpha': 0.0001 * np.arange(1, 100),
    'model__l1_ratio': 0.001 * np.arange(1, 10)
}]
grid_param_krr = [{
    'model__alpha': 0.0001 * np.arange(1, 100),
    'model__degree': [1, 2, 3],
    'model__alpha': [0.6],
    'model__kernel': ['polynomial'],
    'model__coef0': [2.5]
}]

In [63]:
best_params = {
    'Lasso': None,
    'ElasticNet': None,
    'Kernel Ridge': None
}

Grid search is conducted using each pipeline and parameter, and optimal hyperparameters for each model are stored.

In [66]:
search_lasso = GridSearchCV(pipe_lasso, grid_param_lasso, scoring='neg_mean_squared_error', n_jobs=-1).fit(X_train, y_train)
best_params['Lasso'] = search_lasso.best_params_

In [64]:
search_enet = GridSearchCV(pipe_enet, grid_param_enet, scoring='neg_mean_squared_error', n_jobs=-1).fit(X_train, y_train)
best_params['ElasticNet'] = search_enet.best_params_

In [68]:
search_krr = GridSearchCV(pipe_krr, grid_param_krr, scoring='neg_mean_squared_error', n_jobs=-1).fit(X_train, y_train)
best_params['Kernel Ridge'] = search_krr.best_params_

Grid search results are as follows.

In [69]:
best_params

Create a model using grid search results.

In [70]:
model_lasso = Pipeline([
    ('scaler', RobustScaler()),
    ('model', Lasso(alpha=0.0005))
])

model_enet = Pipeline([
    ('scaler', RobustScaler()),
    ('model', ElasticNet(alpha=0.0089, l1_ratio=0.009000000000000001, random_state=3))
])

model_krr = Pipeline([
    ('scaler', RobustScaler()),
    ('model', KernelRidge(alpha=0.6,
                        kernel='polynomial',
                        degree=2,
                        coef0=2.5))
])

The hyperparameters of tree models are diverse and have many combinations. Their optimization takes a lot of time.


So, I looked for XGBoost's hyperparameters using the Optuna package using early stopping and cross validation.

The hyperparameters of tree models are diverse and have many combinations. Their optimization takes a lot of time.

I looked for XGBoost's hyperparameters using the Optuna package using early stopping and cross validation.

Preparations: 'optuna', 'functions-partial', objective function

Pre-understanding:

Optuna is a framework that helps optimize hyperparameters. An objective function is required.

Optuna's objective function selects a new hyperparameter combination of the model every trial.

Optuna's study object is an object that performs optimization. The optimization of the study object requires a partial object and the number of attempts.

The partial object is an object that binds X, y with the objective function to be used by optuna.

Study objects store results that meet the purpose for each trial. Finally, remember the most purposeful hyperparameter combination.

The trial factor of objective embeds the function of specifying the range and value of hyperparameters. It has a hyperparameter name, range or list as a factor in common.
<ol>
    <li>Suggest_int: Select an integer value within the range.</li>
    <li>Suggest_uniform: Select an equal distribution value within a range.</li>
    <li>Suggest_discrete_uniform: Select a discrete uniform distribution value within the range.</li>
    <li>Suggest_loguniform: Select a logarithmic function linear value within a range.</li>
    <li>Suggest_category: Select a value in the list.</li>
</ol>

In [71]:
def objective_xgbr(trial, X, y):
    param = {
        'n_estimators': 2000,
        'max_depth': trial.suggest_int('max_depth', 3, 11),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.005, 0.01),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 100),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 100),
        'subsample': trial.suggest_categorical('subsample', list(np.arange(0.4, 1.1, 0.1))),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', list(np.arange(0.4, 1.1, 0.1))),
        'n_jobs': -1
    }
    model = XGBRegressor(**param)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    train_scores, test_scores = [], []
    for train_idx, test_idx in kf.split(X_train):
        X_tmp_train, X_tmp_test = X_train.iloc[train_idx, :], X_train.iloc[test_idx, :]
        y_tmp_train, y_tmp_test = y_train[train_idx], y_train[test_idx]
        model.fit(X_tmp_train, y_tmp_train,
                 eval_metric=['rmse'], eval_set=[(X_tmp_test, y_tmp_test)],
                 early_stopping_rounds=30, verbose=0,
                 callbacks=[optuna.integration.XGBoostPruningCallback(trial, observation_key='validation_0-rmse')])
        train_score = np.sqrt(mse(y_tmp_train, model.predict(X_tmp_train)))
        test_score = np.sqrt(mse(y_tmp_test, model.predict(X_tmp_test)))
        train_scores.append(train_score)
        test_scores.append(test_score)
    train_score = np.array(train_scores).mean()
    test_score = np.array(test_scores).mean()
    print(f'train score: {train_score}')
    print(f'test score: {test_score}')
    return test_score

In [72]:
optimizer = partial(objective_xgbr, X=X_train, y=y_train)
study = optuna.create_study(direction='minimize')
study.optimize(optimizer, n_trials=100)

We used Optuna to find XGBoost's optimal hyperparameters.

We can check the RMSE by XGBoost's attempt.

In [73]:
optuna.visualization.plot_optimization_history(study)

We can also observe the results for each hyperparameter.

In [74]:
optuna.visualization.plot_slice(study)

In [75]:
# XGBoost HyperParameters
study.best_params

In [76]:
model_xgbr = XGBRegressor(colsample_bytree=0.4, learning_rate=0.00898718134841855, max_depth=8, 
                             n_estimators=2200, reg_alpha=0.036142628805195254, reg_lambda=0.03188665185506858,
                             subsample=0.6, random_state =42)

I haven't done grid search for Gradient Boosting and LightGBM yet (I referred to other notebooks).

In [77]:
model_gbr = gbr(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=5)

In [78]:
model_lgbm = LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

I create a stacking model.

In [79]:
stack_gen = StackingCVRegressor(regressors=(model_lgbm, model_lasso, model_enet, model_krr, model_gbr),
                               meta_regressor=model_xgbr,
                               use_features_in_secondary=True)

#### 3) Model learning.


Cross validation is conducted for all models.

In [80]:
models = [
    model_lasso, model_enet, model_krr, model_gbr, model_xgbr, model_lgbm
]
cross_score = {
    'Lasso': 0,
    'ElasticNet': 0,
    'Kernel Ridge': 0,
    'GradientBoosting': 0,
    'XGBoost': 0,
    'LightGBM': 0,
}

for idx, model in enumerate(models):
    cross_score[list(cross_score.keys())[idx]] = rmsle_cv(model, X_train, y_train).mean()

In [81]:
cross_score

Now, the final learning is conducted using the training data.

In [82]:
for model in models:
    model = model.fit(X_train, y_train)

#### 4) Ensemble: Stacking and blending.

Learn the stacking model.

In [83]:
stack_gen = stack_gen.fit(X_train, y_train)

I weighted each model for the final result. 

I previously gave a higher weight to the model with excellent cross-validation performance.

In [84]:
def blend(X):
    return ((0.15 * model_lasso.predict(X)) + \
            (0.15 * model_enet.predict(X)) + \
            (0.05 * model_krr.predict(X)) + \
            (0.15 * model_xgbr.predict(X)) + \
            (0.15 * model_lgbm.predict(X)) + \
            (0.35 * stack_gen.predict(np.array(X))))

In [85]:
np.sqrt(mse(y_train, blend(X_train)))

In [86]:
np.sqrt(mse(y_train, stack_gen.predict(X_train)))

#### 5) The final result

In [87]:
sub = pd.DataFrame()
sub['Id'] = test_id
sub['SalePrice'] = score = np.expm1(blend(X_test))
sub.to_csv('submission.csv',index=False)